# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from bson.decimal128 import Decimal128
from pymongo import UpdateMany, UpdateOne

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
database_names = mongo.list_database_names()

if "uk_foods" in database_names:
    print("'uk_food' exists.")
else: 
    print("does not exist")


'uk_food' exists.


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_foods']

In [5]:
# review the collections in our new database
collection_names = db.list_collection_names()

for collection_name in collection_names:
    print(collection_name)

establishments


In [6]:
# review a document in the establishments collection
document = db.establishments.find_one()
print(document)

{'_id': ObjectId('654074f193875baa63a6d5ee'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': {'$toInt': '$RatingValue'}, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': Decimal128('1.194762'), 'latitude': Decimal128('51.085797')}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extrac

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data

new_restaurant_data= {
    "BusinessName": "Penang Flavors",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BunsinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200",
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

# Insert the new restaurant into the collection
result = db.establishments.insert_one(new_restaurant_data)
print("Inserted document ID:", result.inserted_id)

Inserted document ID: 65419acddca512be397b30b2


In [9]:
# Check that the new restaurant was inserted
inserted_document = db.establishments.find_one({"BusinessName": "Penang Flavors"})

if inserted_document:
    print("'Penang Flavors' inserted successfully:")
    print(inserted_document)
else:
    print("'Penang Flavors' not found")

'Penang Flavors' inserted successfully:
{'_id': ObjectId('654089bcd3269d5bf7ba94d1'), 'BusinessName': 'Penang Flavors', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BunsinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': Decimal128('1.194762'), 'latitude': Decimal128('51.085797')}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True, 'BusinessTypeID': 1}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [10]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
#define query to find document with businesstype "Restaurant/Cafe/Canteem"
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
limit=1
#define the projection to return only BusinessTypeID and BusinessType fields
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

#excute query and projection
results = db.establishments.find(query, projection).limit(limit)

for doc in results:
    print(doc)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [11]:
# Update the new restaurant with the correct BusinessTypeID
#define filter to identify document
filter = {"BusinessName": "Penang Flavors"}

#define the new data, including businesstypeID
new_data = {
    "$set": {
        "BusinessTypeID": 1
    }
}

#update the document
result = db.establishments.update_one(filter, new_data)

#confirm update sucessful
if result.modified_count > 0:
    print("update successful")
else:
    print("No document matched filter, or update not changed")

No document matched filter, or update not changed


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [12]:
# Find how many documents have LocalAuthorityName as "Dover"
#define query to find "dover"
query = {"LocalAuthorityName": "Dover"}

#count the number of documents that match
count = db.establishments.count_documents(query)

print(f"Number of documents with LocalAuthorityName 'Dover': {count}")

Number of documents with LocalAuthorityName 'Dover': 0


In [13]:
# Delete all documents where LocalAuthorityName is "Dover"
#define query again
query = {"LocalAuthorityName": "Dover"}

#delet all that match dover
result = db.establishments.delete_many(query)

#confirm
print(f"Number of documents deleted: {result.deleted_count}")

Number of documents deleted: 0


In [14]:
# Check if any remaining documents include Dover
#define query once more
query = {"LocalAuthorityName": "Dover"}

#count number of documents
count = db.establishments.count_documents(query)

#display
print(f"Number of remaining documents {count}")

Number of remaining documents 0


In [15]:
# Check that other documents remain with 'find_one'
document = db.establishments.find_one()

#check if any documents were found
if document:
    print("Found at least one document:")
    print(document)
else:
    print("None found")

Found at least one document:
{'_id': ObjectId('654074f193875baa63a6d5ee'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': {'$toInt': '$RatingValue'}, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': Decimal128('1.194762'), 'latitude': Decimal128('51.085797')}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta':

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [16]:
# Change the data type from String to Decimal for longitude and latitude
#define filter to identify documents that need updating
filter = {"geocode.longitude": {"$type": "string"}, "geocode.latitude": {"$type": "string"}}

#define the new value with decimal128
new_value = {
    "$set": {
        "geocode.longitude": Decimal128("1.194762"),
        "geocode.latitude": Decimal128("51.085797")
    }
}

#update all documents that match
result = db.establishments.update_many(filter, new_value)

print(f"Number of documents updated: {result.modified_count}")

Number of documents updated: 1


Use `update_many` to convert `RatingValue` to integer numbers.

In [17]:
count = db.establishments.count_documents({"RatingValue": {"$type": "string"}})

if count > 0:
    print(f"Found {count} documents with string RatingValue")
else:
    print("None")


None


In [18]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [19]:
# Change the data type from String to Integer for RatingValue
# define the filter to idenfity documents where RatingValue is a string
filter = {"RatingValue": {"$type": "string"}}

#define new values with integer data type
new_values = {
    "$set": {
        "RatingValue": {"$toInt": "$RatingValue"}
    }
}

#update all documents that match
result = db.establishments.update_many(filter, new_values)

#confirm update
print(f"Number of documents updated: {result.modified_count}")


Number of documents updated: 0


In [20]:
# Check that the coordinates and rating value are now numbers
#find and print
document = db.establishments.find_one()

#check if found
if document:
    print("Found at least one updated document:")
    print("Longitude:", document["geocode"]["longitude"])
    print("Latitude:", document["geocode"]["latitude"])
    print("RatingValue", document["RatingValue"])
else:
    print("No Updated Documents")


Found at least one updated document:
Longitude: 1.194762
Latitude: 51.085797
RatingValue {'$toInt': '$RatingValue'}
